In [115]:
!pip install pypdf
!pip install bertopic

  Obtaining dependency information for numpy>=1.20.0 from https://files.pythonhosted.org/packages/c0/bc/77635c657a3668cf652806210b8662e1aff84b818a55ba88257abf6637a8/numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl (13.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tables 3.8.0 requires blosc2~=2.0.0, which is not installed.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 4.25.3 which is incompatible.
tensorflow-macos 2.15.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.16.0 which is incompatible.
ibm-watson-machine-learning 1.0.348

In [127]:
from langchain_community.document_loaders import PyPDFLoader, PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from pathlib import Path
import pandas as pd

from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, TextGeneration, OpenAI
import openai





In [58]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len,
    is_separator_regex=False,
)

In [59]:
loader = PyPDFDirectoryLoader("./Data/Raw Data/National Strategy")
docs = loader.load_and_split(text_splitter=splitter)




In [113]:
docs

[Document(page_content='i\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n                        Draft\nNational\n \nArtificial Intelligence Policy\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \nGovernment of Pakistan\nMinistry of Information Technology & Telecommunication\nhttps://moitt.gov.pk', metadata={'source': 'Data/Raw Data/National Strategy/[Pakistan] National AI Policy Consultation Draft V1.pdf', 'page': 0}),
 Document(page_content='ii \n Acknowledgments  \nThe Government of Pakistan, Ministry of IT & Telecom , pays its gratitude to all the officials  and consultants, \nparticularly  RSM Pakistan and GlowBug Technologies (Pvt.) Ltd. , facilitators, developers, and stakeholders who \nrigorously and relentlessly participated in the revi ew, drafting, harmonizing, and ratification of the National \nArtificial Intelligence Policy – 2022, helping the Ministry with an all -inclusive user -centric, evidence -based, \nforward -lookin

In [114]:
print(docs[10].page_content)

5 
 Foreword  
Through its continued efforts toward  the early realization of the Digital Pakistan vision of the Government 
of Pakistan, the Ministry of IT & Telecom is committed to providing  its people with timely and equal access 
to opportunities by stimulating a culture of innovation through an overarching de velopmental agenda 
orchestrated to embrace cutting -edge technologies such as Artificial Intelligence efficiently and 
responsibly.   
In this regard, the  Artificial Intelligence  (AI) Policy  is a pivotal milestone for  transforming Pakistan into a  
knowledge -based economy as it spells out a national strategy  to establish an ecosystem necessary for AI 
adoption by harnessing an agile framework for addressing different aspects of unique user journeys 
encompassing different market horizontals and industry verticals by ensuring responsible use of AI.  
Furthermore, the policy aims to go beyond the meagre approach of adopting technology to fundamentally


In [98]:
def documents2Dataframe(documents) -> pd.DataFrame:
    import uuid

    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df

In [163]:
df_sampling = documents2Dataframe(docs)


In [164]:
df_sampling

,text,source,page,chunk_id
0,i\n \n \n \n \n \n \n \n \n \n \n \n \n \n \n ...,Data/Raw Data/National Strategy/[Pakistan] Nat...,0,f147e79086a94b62b227f44a38683469
1,ii \n Acknowledgments \nThe Government of Pak...,Data/Raw Data/National Strategy/[Pakistan] Nat...,1,a2937d76d8da4f2298e1ba47749caa5f
2,iii \n Table of Contents \n1 Executive Summar...,Data/Raw Data/National Strategy/[Pakistan] Nat...,2,224bedaa5ca54266a7482b65c1fa455a
3,3.2 Scope ................................ .....,Data/Raw Data/National Strategy/[Pakistan] Nat...,2,ed003937d7304ea6bd0beb8aad174498
4,4.1 1st Pillar: AI Market Enablement ...........,Data/Raw Data/National Strategy/[Pakistan] Nat...,2,5385d702de704c4494d010f2df2f3502
...,...,...,...,...
1438,"Joshua Gans, Avi Goldfarb https://www.amazon.c...",Data/Raw Data/National Strategy/[Qatar] Nation...,17,51eda1ee66374804afa02b75fe7c2e28
1439,Oun application for municipalities services ht...,Data/Raw Data/National Strategy/[Qatar] Nation...,17,ccbef1557ab7470a9785b356f5a0f552
1440,in the context of local cultural norms. For ex...,Data/Raw Data/National Strategy/[Qatar] Nation...,17,04d43c004a0e4cdf8101d6c53228fbdb
1441,19\nNational Artificial Intelligence Strategy ...,Data/Raw Data/National Strategy/[Qatar] Nation...,18,cd0f1843db124c29b6548d3d1481a338


In [111]:
df_sampling['text'][141]

'making them experiment with using AI -based models in a controlled yet agile environment.   \nIII. AI a doption would disrupt how work will be done in public and private sectors and various \nindustries, directly affecting the country ’s socio -economic fabric. People working in traditional \nways and not skilled in AI -based technologies are at risk of losing thei r jobs if not pivoted timely.  \nThis issue must  be addressed on a priority basis, which can only be done through awareness and \nupskilling.  Customized learning programs, workshops, and courses can assist such individuals in \nsurviving this transition and becom ing an integral part of digitally transformed workplaces.  \nIV. According to the surveys, at a sectoral scale, half of the respondents are well versed with AI and \nrelated technologies, while the other half possess working knowledge. To teach an AI-based \ncurricu lum, the survey findings point to including short courses and 6 -12 month -long boot'

#### Data Extraction - Third experiment - Set of sampling

In [77]:
## Input data directory
data_dir = "National Strategy"
inputdirectory = Path(f"./Data/Raw Data/{data_dir}")

## This is where the output csv files will be written
out_dir = "Intermediate Data"
outputdirectory = Path(f"./Data/{out_dir}")

In [78]:
# Directory containing the .pdf files
directory = "./Data/Raw Data/National Strategy"

# List to store loaded documents
documents = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith(".pdf"):
        file_path = os.path.join(directory, filename)
        loader = PyPDFLoader(file_path)
        documents.extend(loader.load())

# Now, the 'documents' list contains all the documents loaded from the .txt files in the directory.

In [80]:
len(documents)

556

In [ ]:
def documents2Dataframe(documents) -> pd.DataFrame:
    import uuid

    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            **chunk.metadata['source'],
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df

## Topic Modeling - BERTopic

#### Embeddings

In [118]:
# Convert our column into a list
texts = list(df_sampling['text'].values)


In [122]:
# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-mpnet-base-v2")


In [123]:
embeddings = embedding_model.encode(texts, show_progress_bar=True)

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

In [171]:
embeddings

array([[-7.3423413e-05,  4.8945505e-02, -3.6187608e-02, ...,
         1.3831359e-02, -3.5541855e-02, -2.0980146e-02],
       [ 4.7136318e-02,  4.3284182e-02, -3.9959706e-02, ...,
         3.2031696e-02, -4.6796147e-02, -8.2864696e-03],
       [ 1.7579101e-02, -1.5924098e-02, -5.5648666e-02, ...,
         1.9876931e-02, -7.2451960e-03,  5.0753085e-03],
       ...,
       [ 1.1809425e-02,  6.2407423e-02, -3.6552388e-02, ...,
         2.8599052e-02,  3.1834912e-02,  6.4921891e-03],
       [ 2.0708462e-02,  4.3231890e-02, -2.3956854e-02, ...,
         1.4826042e-02,  7.6147681e-03,  2.0427229e-03],
       [-1.1085893e-03, -1.2075085e-02, -4.8567716e-02, ...,
         7.2799865e-03, -1.6352529e-02, -2.6653156e-02]], dtype=float32)

#### Dimensionality Reduction, Clustering and Vectorizer

In [134]:
umap_model = UMAP(n_neighbors=25, n_components=5, min_dist=0.0, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=2, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
vectorizer_model = CountVectorizer(stop_words="english", min_df=2, ngram_range=(1, 3))

#### Representation models

In [128]:

# KeyBERT
keybert_model = KeyBERTInspired()

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# # GPT-3.5
# prompt = """
# I have a topic that contains the following documents:
# [DOCUMENTS]
# The topic is described by the following keywords: [KEYWORDS]

# Based on the information above, extract a short but highly descriptive topic label of at most 5 words. Make sure it is in the following format:
# topic: <topic label>
# """
# client = openai.OpenAI(api_key=read_key_from_file())
# openai_model = OpenAI(client, model="gpt-3.5-turbo", exponential_backoff=True, chat=True, prompt=prompt)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    #"OpenAI": openai_model,  # Uncomment if you will use OpenAI
    "MMR": mmr_model
}

#### Training

In [135]:
topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,  # while we use pre-computed embeddings, internally BERTopic still needs this parameter
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True
)

# Train model
topics, probs = topic_model.fit_transform(texts, embeddings)

2024-05-24 13:02:56,945 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-24 13:03:00,028 - BERTopic - Dimensionality - Completed ✓
2024-05-24 13:03:00,029 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-24 13:03:00,065 - BERTopic - Cluster - Completed ✓
2024-05-24 13:03:00,069 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-24 13:04:40,267 - BERTopic - Representation - Completed ✓


#### Visualizations

In [139]:
# Show topics
freq = topic_model.get_topic_info(); freq.head(15)

,Topic,Count,Name,Representation,KeyBERT,MMR,Representative_Docs
0,-1,334,-1_data_ai_model_organisations,"[data, ai, model, organisations, ai applicatio...","[ethical ai, ethical ai framework, ai model, a...","[data, ai, model, organisations, ai applicatio...",[ETHICAL AI FRAMEWORK APPENDIX \n \n6-16 \n ...
1,0,45,0_qatar_ai technology_businesses_guidelines,"[qatar, ai technology, businesses, guidelines,...","[national ai strategy, national artificial int...","[qatar, ai technology, businesses, guidelines,...",[5\nNational Artificial Intelligence Strategy ...
2,1,43,1_education_sw_sw ai_curriculum,"[education, sw, sw ai, curriculum, schools, sc...","[ai education, ai curriculum, ai allied techno...","[education, sw, sw ai, curriculum, schools, sc...","[customized department), and Industrial AI Pro..."
3,2,29,2_philippines_ai strategy roadmap_strategy roa...,"[philippines, ai strategy roadmap, strategy ro...","[national ai strategy, ai strategy roadmap, na...","[philippines, ai strategy roadmap, strategy ro...",[NATIONAL AI STRATEGY ROADMAP FOR THE PHILIPPI...
4,3,29,3_pakistan_policy_ai policy_ai adoption,"[pakistan, policy, ai policy, ai adoption, sec...","[ai based technologies, national artificial in...","[pakistan, policy, ai policy, ai adoption, sec...",[sharing standardized data will ensure the ena...
5,4,24,4_society_social_social principles_human centric,"[society, social, social principles, human cen...","[social principles ai, society ai, ai ready so...","[society, social, social principles, human cen...",[7 \n 4 Social Principles of Human -Centric A...
6,5,23,5_korea_korean_artificial_strategy,"[korea, korean, artificial, strategy, intellig...","[ai world leader, artificial intelligence poli...","[korea, korean, artificial, strategy, intellig...",[21\n2. Where is Korea aiming? (Vision and Goa...
7,6,18,6_india_consultations_paper_ai systems,"[india, consultations, paper, ai systems, prin...","[ai ethics, responsible ai, ai india, artifici...","[india, consultations, paper, ai systems, prin...",[improving access and quality and higher effic...
8,7,17,7_job_workforce_changes_automation,"[job, workforce, changes, automation, awarenes...","[ai, skills ai, job automation, basic skills a...","[job, workforce, changes, automation, awarenes...","[Global companies such as Microsoft, Amazon, A..."
9,8,16,8_roles_project_responsibilities_roles respons...,"[roles, project, responsibilities, roles respo...","[ai governance structure, ai governance, ethic...","[roles, project, responsibilities, roles respo...",[their corresponding responsibilities . \n \n...


In [148]:
topic_model.get_topic(0, full=True)  # Select the most frequent topic

{'Main': [('qatar', 0.05982546440352836),
  ('ai technology', 0.012312849799269034),
  ('businesses', 0.011800167422276846),
  ('guidelines', 0.010257391343404351),
  ('world', 0.009444092587170239),
  ('strategy', 0.008981548866182259),
  ('gov qa', 0.008906928435222835),
  ('local', 0.008872691665470058),
  ('language', 0.008811053514518646),
  ('qa', 0.008718900920147907)],
 'KeyBERT': [('national ai strategy', 0.6385046),
  ('national artificial intelligence', 0.57856965),
  ('ai future', 0.5761137),
  ('national ai', 0.57600844),
  ('ai technology', 0.5291699),
  ('ai', 0.5124967),
  ('modern ai', 0.5113133),
  ('ai solutions', 0.5000566),
  ('ai based', 0.47077513),
  ('ai enabled', 0.4680573)],
 'MMR': [('qatar', 0.05982546440352836),
  ('ai technology', 0.012312849799269034),
  ('businesses', 0.011800167422276846),
  ('guidelines', 0.010257391343404351),
  ('world', 0.009444092587170239),
  ('strategy', 0.008981548866182259),
  ('gov qa', 0.008906928435222835),
  ('local', 0.00

In [143]:
topic_model.visualize_topics()


In [147]:
topic_model.topic_representations_

{-1: [('data', 0.004711170867681569),
  ('ai', 0.0046521437913720634),
  ('model', 0.004365704358676696),
  ('organisations', 0.00408052819013431),
  ('ai application', 0.003887949739809465),
  ('application', 0.0037846240457796256),
  ('ethical', 0.0035796752448587054),
  ('ensure', 0.0035776489355264078),
  ('process', 0.003497337450093672),
  ('applications', 0.003384670635564553)],
 0: [('qatar', 0.05982546440352836),
  ('ai technology', 0.012312849799269034),
  ('businesses', 0.011800167422276846),
  ('guidelines', 0.010257391343404351),
  ('world', 0.009444092587170239),
  ('strategy', 0.008981548866182259),
  ('gov qa', 0.008906928435222835),
  ('local', 0.008872691665470058),
  ('language', 0.008811053514518646),
  ('qa', 0.008718900920147907)],
 1: [('education', 0.029263260672741716),
  ('sw', 0.022714438251729814),
  ('sw ai', 0.020810223448869774),
  ('curriculum', 0.016793457761493002),
  ('schools', 0.01613701491805157),
  ('school', 0.01585348827627249),
  ('training', 0

In [161]:
topic_model.visualize_barchart(top_n_topics=20)